In [1]:
import sys
import os
from os import path
current_folder = path.dirname(path.abspath('')) 
sys.path.append(current_folder)
from estimators import *
from geomle import geomle, mle, DataGenerator
import multiprocessing as mp
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from scipy.linalg import cholesky
from scipy.special import gammainc, lambertw
import scipy.io
import matplotlib as mpl
from matplotlib import pyplot as plt
import umap
import seaborn as sns
import random
import time
import numpy as np
import pandas as pd
import pickle
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages
from functools import wraps
import subprocess
from IPython.display import display_html
from operator import itemgetter
ig0 = itemgetter(0)
ig1 = itemgetter(1)
ig2 = itemgetter(2)
rpy2.robjects.numpy2ri.activate()
utils = rpackages.importr('utils')
#utils.install_packages('intrinsicDimension')
#utils.install_packages('ider')
intdimr = rpackages.importr('intrinsicDimension')
ider   = rpackages.importr('ider')
r_base = rpackages.importr('base')

In [2]:
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def mean_sqe(estimations, truth):
    '''
    Mean squared error 
    '''
    return ((estimations - truth)^2/truth).sum() /len(truth) 
    
def mean_pe(estimations, truth):
    '''
    Mean percentage error 
    '''
    return (abs(estimations - truth)/truth).sum() /len(truth)*100

def mean_ge(estimations, truth):
    '''
    Mean geometric error: The geometric mean of the error *ratio*. It is always >= 1.
    '''
    ratios = np.concatenate(((estimations/truth)[np.newaxis, :], (truth/estimations)[np.newaxis, :]), axis=0)
    return np.power(ratios.max(axis=0).prod(), 1.0/len(estimations))

def med_pe(estimations, truth):
    '''
    Median error in %.
    '''
    return np.percentile(abs(estimations - truth)/truth, q=50)*100


def randball(n_points,ndim,radius,center = []):
    ''' Generate uniformly sampled ndim-sphere interior'''
    if center == []:
        center = np.array([0]*ndim)
    r = radius
    x = np.random.normal(size=(n_points, ndim))
    ssq = np.sum(x**2,axis=1)
    fr = r*gammainc(ndim/2,ssq/2)**(1/ndim)/np.sqrt(ssq)
    frtiled = np.tile(fr.reshape(n_points,1),(1,ndim))
    p = center + np.multiply(x,frtiled)
    return p, center

def proxy(tup):
    function,X,Dict = tup
    return function(X,**Dict)

def get_nn(X,k,n_jobs=1):
    neigh = NearestNeighbors(n_neighbors=k,n_jobs=n_jobs)
    neigh.fit(X)
    dists, inds = neigh.kneighbors(return_distance=True)
    return dists,inds

def asPointwise(data,function, params, precomputed_knn = None, n_neighbors=100, n_jobs=1):
    '''Use a global estimator as a pointwise one by creating kNN neighborhoods'''
    if precomputed_knn is not None:
        knn = precomputed_knn
    else:
        _, knn = get_nn(data, k=n_neighbors, n_jobs=n_jobs)
        
    if n_jobs > 1:
        pool = mp.Pool(n_jobs)
        results = pool.map(proxy,[(function,data[i,:],params) for i in knn])
        pool.close()
        return results
    else:
        return [function(data[i,:],**params) for i in knn]


from functools import wraps
def calculate_time(func): 
    @wraps(func)
    def inner_func(*args, **kwargs): 
        begin = time.time() 
        res = func(*args, **kwargs) 
        end = time.time()
        return res, end - begin
    return inner_func

class DimEst():
    def __init__(self):
        self.names = ['MLE', 'GeoMLE', 'MIND', 'DANCo', 'FastDANCo', 'ESS', 'PCA', 'CD','FisherS','ANOVA','TwoNN']
        self.caldatas = {}
        
    def estimateAllMethods(self, data):
        dim = data.shape[1]
        self.funcs = {'MLE':          self.mle(data),
                      #'GeoMLE':       self.geomle(data, dim),
                      'MiND':         self.mind_mlk(data, dim),
                      #'DANCo':        self.danco(data, dim),
                      'FastDANCo':    self.fast_danco(data),
                      'ESS':          self.ess(data),
                      #'PCA':          self.pca(data),
                      #'CD':           self.cd(data),
                      'FisherS':      self.fisherS(data),
                      'ANOVA':        self.anova(data),
                      'TwoNN':        self.twonn(data)
                     }
                      
        self.times = {key: ig1(val) for key, val in self.funcs.items()}
        self.funcs = {key: ig0(val) for key, val in self.funcs.items()}
        return self.funcs, self.times
    
    def estimateAllMethodsLocally(self, data, k, n_jobs = 1):
        dim = data.shape[1]
        
        _, knn = get_nn(data, k, n_jobs)
        
        self.funcs = {'MLE':          self.mlelocal(data,k),
                      #'GeoMLE':       self.geomle(data, dim),
                      'MiND':         asPointwise(data,self.mind_mlk,{'dim':dim},precomputed_knn=knn,n_jobs=1),
                      #'DANCo':        asPointwise(data,self.danco,{'dim':dim},precomputed_knn=knn,n_jobs=1),
                      'FastDANCo':    self.fast_dancoloop(data),
                      'ESS':          asPointwise(data,self.ess,{},precomputed_knn=knn,n_jobs=1),
                      #'PCA':          self.pca(data),
                      #'CD':           self.cd(data),
                      'FisherS':      asPointwise(data,self.fisherS,{},precomputed_knn=knn,n_jobs=n_jobs),
                      'ANOVA':        self.anovalocal(data,k),
                      'TwoNN':        asPointwise(data,self.twonn,{},precomputed_knn=knn,n_jobs=n_jobs)
                     }
                      
        self.times = {}
        for key, val in self.funcs.items():
            if key in ['MLE','ANOVA','FastDANCo']:
                self.funcs[key] = np.array(val[0])
                self.times[key] = val[1]
            else:
                self.funcs[key] = np.array([i[0] for i in val])
                self.times[key] = np.sum([i[1] for i in val])
            
        return self.funcs, self.times
    
    @staticmethod
    @calculate_time
    def mle(data):
        return intdimr.maxLikGlobalDimEst(data,k=20).rx2('dim.est')[0]
    
    @staticmethod
    @calculate_time
    def mlelocal(data,k):
        res = intdimr.maxLikPointwiseDimEst(data,k=k)
        return np.array([i[0] for i in res])

    @staticmethod
    @calculate_time
    def geomle(data, dim):
#         k1 =  k1_log(dim)
#         k2 =  k2_log(dim)
        return geomle(pd.DataFrame(data), k1=20, k2=55, nb_iter1=1, alpha=5e-3).mean()
    
    @staticmethod
    @calculate_time
    def mind_mlk(data, dim):
        return intdimr.dancoDimEst(data, k=10, D=min(dim,100), ver="MIND_MLk").rx2('dim.est')[0]
    
    #@staticmethod
    @calculate_time
    def danco(self,data, dim):
        try:
            res = intdimr.dancoDimEst(data, k=10, D=min(dim,100), calibration_data = self.caldatas[len(data)], ver="DANCo")
            self.caldatas[len(data)]=res[2]
            return res.rx2('dim.est')[0]
        except:
            res = intdimr.dancoDimEst(data, k=10, D=min(dim,100), ver="DANCo")
            self.caldatas[len(data)]=res[2]
            return res.rx2('dim.est')[0]

    @staticmethod
    @calculate_time
    def fast_danco(data):
        return runDANCo(data)[0]
    
    @staticmethod
    @calculate_time
    def fast_dancoloop(data):
        return runDANColoop(data)[0]
    
    @staticmethod
    @calculate_time
    def ess(data):
        return ess_py[0]
    
    @staticmethod
    @calculate_time
    def pca(data):
        return intdimr.pcaLocalDimEst(data, 'FO').rx2('dim.est')[0]
    
    @staticmethod
    @calculate_time
    def cd(data):
        return corint_py(data, k1=10, k2=20)[0]
    
    @staticmethod
    @calculate_time
    def fisherS(data):
        return SeparabilityAnalysis(data,ProducePlots=0,alphas=np.arange(.2,1,.02)[None])[1][0]
    
    @staticmethod
    @calculate_time
    def anova(data):
        return runANOVAglobal(data)[0,0]
    
    @staticmethod
    @calculate_time
    def anovalocal(data,k):
        return runANOVAlocal(data,k=k)[:,0]
    
    @staticmethod
    @calculate_time
    def twonn(data):
        res = twonn_py(data)
        return res    

In [3]:
path_to_estimators='/home/utilisateur/id-concentration/estimators'
path_to_matlab='matlab'

In [4]:
def runDANCo_stats(data,k=10,path_to_estimators=path_to_estimators, path_to_matlab = path_to_matlab):
    ''' Run matlab script from shell '''
    path_to_estimators += '/idEstimation/idEstimation/'
    np.savetxt(path_to_estimators+'temp_data2.txt',data)
    subprocess.call([path_to_matlab,"-nodisplay", "-nosplash", "-nodesktop","-nojvm","-r",
                        "data=dlmread('"+path_to_estimators+"/temp_data2.txt');cd ('"+path_to_estimators+"');k="+str(k)+";[d,kl,dHat,mu,tau,mu2,tau2] = DANCoFit(data',k);save('variables2.mat');exit;"""])
    return scipy.io.loadmat(path_to_estimators+'variables2.mat')

def runANOVAlocal_stats(data,k=None,path_to_estimators=path_to_estimators, path_to_matlab = path_to_matlab):
    ''' Run matlab script from shell '''
    path_to_estimators += '/ANOVA_dimension_estimator-master/'
    if k is None:
        #ANOVA default
        k = min(500,max(round(10*np.log10(data.shape[0])), 10))
    np.savetxt(path_to_estimators+'/temp_data.txt',data)
    subprocess.call([path_to_matlab,"-nodisplay", "-nosplash", "-nodesktop","-nojvm","-r",
                        "data=dlmread('"+path_to_estimators+"/temp_data.txt');cd ('"+path_to_estimators+"');k="+str(k)+";[estimates, first_moments,indicesknn, Us]=ANOVA_local_estimator(data',data','k',k,'maxdimension',100);save('variables.mat');exit"""])
    return scipy.io.loadmat(path_to_estimators+'variables.mat')

In [5]:
x=np.random.random((100,10))

In [27]:
DANCostats = runDANCo_stats(x)
ANOVAstats = runANOVAlocal_stats(x)

In [32]:
DANCostats['dHat'],DANCostats['tau'],DANCostats['mu2'],DANCostats['tau2']

(array([[7.30488281]]), array([[14.99826246]]), array([[1.26333414],
        [1.18369071],
        [1.46184059],
        [1.45420724],
        [1.36779072],
        [1.23116489],
        [1.292453  ],
        [1.4276694 ],
        [1.52309895],
        [1.00434938],
        [1.31558813],
        [1.33823173],
        [1.25365911],
        [1.15750047],
        [1.26497099],
        [1.50862823],
        [1.04518781],
        [1.28057527],
        [1.4480288 ],
        [1.40057257],
        [1.46266693],
        [1.2255435 ],
        [1.31855321],
        [1.13490239],
        [0.96226026],
        [1.36601671],
        [1.13198798],
        [1.35310881],
        [1.22087496],
        [1.30977988],
        [1.17442854],
        [1.15315881],
        [1.03048458],
        [1.41479139],
        [1.30330253],
        [1.16935075],
        [1.56560175],
        [1.36001142],
        [1.40656316],
        [1.43977358],
        [1.09278965],
        [1.22932407],
        [1.51220686],
       

In [7]:
ANOVAstats = runANOVAlocal_stats(x)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon Jan 13 18:29:11 2020',
 '__version__': '1.0',
 '__globals__': [],
 'Us': array([[0.21808787],
        [0.11005293],
        [0.19795582],
        [0.19742206],
        [0.15264546],
        [0.1348864 ],
        [0.16789051],
        [0.21350423],
        [0.1931573 ],
        [0.27433333],
        [0.24477987],
        [0.27679024],
        [0.12387628],
        [0.15029736],
        [0.22559329],
        [0.153663  ],
        [0.18843839],
        [0.20448779],
        [0.11230502],
        [0.25341871],
        [0.1311321 ],
        [0.21718965],
        [0.24593992],
        [0.23927625],
        [0.31238416],
        [0.29870399],
        [0.25530637],
        [0.11319938],
        [0.20935083],
        [0.16287908],
        [0.14653958],
        [0.26173277],
        [0.18074732],
        [0.2792503 ],
        [0.21270104],
        [0.15302143],
        [0.34068704],
        [0.27500575],
        [0.1637712 

### Testing for uniformity

In [63]:
def compute_kl_divergence(p_probs, q_probs):
    """"KL (p || q)"""
    kl_div = p_probs * np.log(p_probs / q_probs)
    return np.sum(kl_div)

def KLdivergence(x, y):
    """ 
    Compute the Kullback-Leibler divergence between two multivariate samples.
    Parameters
    ----------
    x : 2D array (n,d)
    Samples from distribution P, which typically represents the true
    distribution.
    y : 2D array (m,d)
    Samples from distribution Q, which typically represents the approximate
    distribution.
    Returns
    -------
    out : float
    The estimated Kullback-Leibler divergence D(P||Q).
    References
    ----------
    Pérez-Cruz, F. Kullback-Leibler divergence estimation of
    continuous distributions IEEE International Symposium on Information
    Theory, 2008.
    """
    from scipy.spatial import cKDTree as KDTree

    # Check the dimensions are consistent
    x = np.atleast_2d(x)
    y = np.atleast_2d(y)

    n,d = x.shape
    m,dy = y.shape
    print(x.shape,y.shape)

    assert(d == dy)


    # Build a KD tree representation of the samples and find the nearest neighbour
    # of each point in x.
    xtree = KDTree(x)
    ytree = KDTree(y)

    # Get the first two nearest neighbours for x, since the closest one is the
    # sample itself.
    r = xtree.query(x, k=2, eps=.01, p=2)[0][:,1]
    s = ytree.query(x, k=1, eps=.01, p=2)[0]

    # There is a mistake in the paper. In Eq. 14, the right side misses a negative sign
    # on the first term of the right hand side.
    return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))

In [69]:
print('KL(True||Uniform): ', compute_kl_divergence(x, y))
print('KL(True||Binomial): ', compute_kl_divergence(x, y2))

KL(True||Uniform):  nan
KL(True||Binomial):  nan


In [50]:
import numpy as np
from universal_divergence import estimate

mean = [0, 0]
cov = [[1, 0], [0, 10]]
x = np.random.multivariate_normal(mean, cov, 100)
y = np.random.multivariate_normal(mean, cov, 100)
print(estimate(x, y))  # will be close to 0.0

mean2 = [10, 0]
cov2 = [[5, 0], [0, 5]]
z = np.random.multivariate_normal(mean2, cov2, 100)
print(estimate(x, z))  # will be bigger than 0.0

0.3542997730631076
4.83810981976497


In [65]:
x=randsphere(100,3,1)[0]
y=randsphere(100,3,1)[0]

In [62]:
x=randsphere(100,3,1)[0]
y=randsphere(100,3,1)[0]
y2=randsphere(100,3,1)[0] + np.random.normal(size=(100,3))

estimate(x,y),estimate(x,y2)

(0.006824791827676679, 1.5015690350957318)